# Analysis of Results
Note, this notebook requires the file `H2_exp_output.json` (which is created in the `2_efficient_graph.ipynb` notebook) to exist.

Here we analyse some of the results from the DASK run for the H2 experiment.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import qpfas

## Load `json` file

In [ ]:
exp_df = pd.read_json ("H2_exp_output.json")
exp_df.head()

## Compute VQE Error

In [ ]:
vqe_energies = np.array([i["converged_energy"] for i in exp_df["vqe_output"].values])

exp_df["vqe_error_abs"] = np.abs(vqe_energies - exp_df["fci_energy"])
exp_df["hf_error"] = exp_df["hf_energy"] - exp_df["fci_energy"]


## Explore Data

In [ ]:
%matplotlib notebook

plt.figure(figsize=(10, 6))

exp_vars = [f"{i['transformation']}\n{i['optimizer']}" for _, i in exp_df.iterrows()]

exp_df["vqe_error_abs"].plot(marker='o', label='VQE')
plt.xticks(np.arange(len(exp_vars)), exp_vars)

plt.ylabel("Error (H)")
plt.yscale("log")
plt.axhline(y=exp_df["hf_error"].values[0], c="k", ls='dashed', label='HF')
plt.legend()
plt.tight_layout()
plt.show()

## Graph Reconstruction
We can resconstruct the orginal dask dag from the results

In [ ]:
graph = qpfas.workflow.reconstruct_graph(exp_df["node_history"].values)
qpfas.workflow.DaskDAG.visualize_dag(graph)